In [10]:
import pandas as pd
import nltk
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
import string

In [50]:
#load top_10k_mallet.csv with sub topics
lda_mallet = pd.read_csv('/Users/jcheah/Documents/MITB/Term 2/Text analytics/Project/final sentiment analysis/top_10k_mallet.csv')
#load english dataset with around 10k rows
top_10k = pd.read_csv('/Users/jcheah/Documents/MITB/Term 2/Text analytics/Project/top10k_en.csv')


In [118]:
#Standard Sentiment Analysis

#load pos_lexicon and neg_lexicon for standard sentiment analysis
pos_lexicon = '/Users/jcheah/Documents/MITB/Term 2/Text analytics/Week8/Lab8-stu/positive-words.txt'
neg_lexicon = '/Users/jcheah/Documents/MITB/Term 2/Text analytics/Week8/Lab8-stu/negative-words.txt'

#subset the required columns
top_10k_standard = top_10k.loc[:, ['name','business_id', 'text']]

#split the reviews into texts by space
top_10k_standard['text_sep'] = top_10k_standard['text'].map(lambda x:x.split())

#create sentiment lexicons
def create_dictionary(path):
 
    dictionary = {}
    f = open(path, 'r', encoding = "ISO-8859-1")
    for line in f:
        line = line.strip()
        dictionary[line] = 1

    f.close()
    return dictionary
    


#calculate sentiment score
def sentiment_score(dictionary):

    score = top_10k_standard['text_sep'].map(lambda row: list(map(lambda x: 1 if x in dictionary else 0, row)))
    score_list = []

    for row in score:
        score_list.append(sum(row))
        
    score_df = pd.DataFrame(score_list)
    return score_df



In [ ]:
#create positive and negative lexicons 
pos_dict = create_dictionary(pos_lexicon)
neg_dict = create_dictionary(neg_lexicon)

#calculate sentiment score for each review
top_10k_standard['pos_score'] = sentiment_score(pos_dict)
top_10k_standard['neg_score'] = sentiment_score(neg_dict)

top_10k_standard['topic_score'] = top_10k_standard['pos_score'] / (top_10k_standard['pos_score'] + abs(top_10k_standard['neg_score']))*5


In [126]:
def final_score_standard(dataset):

    reviews_score = dataset['topic_score']
    reviews_score = pd.DataFrame(reviews_score)
    
    #add new columns to lda_mallet to have individual review score
    lda_mallet['topic_score'] = reviews_score
    top_10k_score = lda_mallet.loc[:, ['name', 'topic', 'topic_score']]
    
    #calculate the average ratings for each topic
    top_10k_score = round(abs((top_10k_score.groupby(['name',  'topic']).sum() /
                    (top_10k_score.groupby(['name', 'topic']).count() + 1))), 1)
    
    pd.set_option('display.max_rows', 100)
    return top_10k_score

final_score_standard = final_score_standard(top_10k_standard)
final_score_standard



topic_score
name                                 topic                    
"Banh Mi Boys"                       Atmosphere            4.0
                                     Food                  3.9
                                     Price                 3.5
                                     Service               3.9
                                     Waiting time          3.7
"Basil Thai Kitchen"                 Atmosphere            1.7
                                     Food                  2.5
                                     Price                 2.9
                                     Service               3.6
                                     Waiting time          2.2
"Black Hoof"                         Atmosphere            3.9
                                     Food                  3.9
                                     Price                 3.8
                                     Service               4.0
                                     Waiting time          3.8
"Bonjour Brioche"                    Atmosphere            2.9
                                     Food                  3.5
                                     Price                 3.8
                                     Service               3.1
                                     Waiting time          3.3
"Brassaii"                           Atmosphere            3.6
                                     Food                  4.1
                                     Price                 3.1
                                     Service               3.8
                                     Waiting time          3.9
"Chino Locos Original"               Atmosphere            3.6
                                     Food                  3.6
                                     Price                 3.4
                                     Service               3.6
                                     Waiting time          3.7
"Ciao Wine Bar"                      Atmosphere            3.0
                                     Food                  3.4
                                     Price                 4.0
                                     Service               3.8
                                     Waiting time          3.7
"Circus Coffee House"                Food                  3.7
                                     Service               1.7
"District"                           Atmosphere            3.4
                                     Food                  3.6
                                     Price                 1.5
                                     Service               3.5
                                     Waiting time          2.9
"Eggspectation Resto-Cafe"           Atmosphere            2.2
                                     Food                  3.0
                                     Service               0.0
                                     Waiting time          3.8
"Elegance Chinese Cuisine & Banquet" Atmosphere            2.6
                                     Food                  3.3
                                     Price                 3.7
                                     Service               3.7
...                                                        ...
"Schwartz's"                         Service               2.9
                                     Waiting time          3.1
"Spuntini Restaurant & Bar"          Atmosphere            3.6
                                     Food                  2.3
                                     Price                 1.0
                                     Service               3.2
                                     Waiting time          3.3
"Sumo Ramen"                         Atmosphere            3.3
                                     Food                  2.3
                                     Price                 3.6
                                     Service               3.3
                   

In [41]:
# Vader sentiment analysis
def sentiment_analyzer_scores(review, nth):
    score_list = []
    for i in range(nth):
        score = analyzer.polarity_scores(review[i])
        for cat, point in score.items():
            #only take compound score
            if cat == 'compound':
                score_list.append(point)
    return score_list


In [120]:
top_10k = pd.read_csv('/Users/jcheah/Documents/MITB/Term 2/Text analytics/Project/top10k_en.csv')

def final_score_vader(dataset):
    
    reviews = dataset['text'] 
    reviews_score = sentiment_analyzer_scores(reviews, len(reviews))
    reviews_score = pd.DataFrame(reviews_score)
    lda_mallet['topic_score'] = reviews_score
    top_10k_score = lda_mallet.loc[:, ['name', 'topic', 'topic_score']]
    
    #multiply the score by 5 to comply with the rating scale
    top_10k_score = round(abs((top_10k_score.groupby(['name',  'topic']).sum() /
                    top_10k_score.groupby(['name', 'topic']).count()) * 5), 1)
    
    pd.set_option('display.max_rows', 100)
    top_10k_score
    return top_10k_score

final_score_vader = final_score_vader(top_10k)
final_score_vader


topic_score
name                                 topic                    
"Banh Mi Boys"                       Atmosphere            2.7
                                     Food                  3.4
                                     Price                 3.4
                                     Service               3.7
                                     Waiting time          3.6
"Basil Thai Kitchen"                 Atmosphere            4.9
                                     Food                  4.8
                                     Price                 3.6
                                     Service               3.5
                                     Waiting time          4.6
"Black Hoof"                         Atmosphere            4.1
                                     Food                  3.8
                                     Price                 3.5
                                     Service               4.2
                                     Waiting time          3.8
"Bonjour Brioche"                    Atmosphere            4.4
                                     Food                  3.1
                                     Price                 3.9
                                     Service               4.7
                                     Waiting time          2.6
"Brassaii"                           Atmosphere            3.4
                                     Food                  3.9
                                     Price                 4.1
                                     Service               4.1
                                     Waiting time          3.9
"Chino Locos Original"               Atmosphere            4.2
                                     Food                  3.3
                                     Price                 3.0
                                     Service               3.0
                                     Waiting time          3.8
"Ciao Wine Bar"                      Atmosphere            2.7
                                     Food                  4.0
                                     Price                 4.4
                                     Service               3.5
                                     Waiting time          2.9
"Circus Coffee House"                Food                  3.6
                                     Service               4.9
"District"                           Atmosphere            3.9
                                     Food                  3.5
                                     Price                 2.3
                                     Service               3.0
                                     Waiting time          3.0
"Eggspectation Resto-Cafe"           Atmosphere            4.8
                                     Food                  2.8
                                     Service               0.0
                                     Waiting time          3.9
"Elegance Chinese Cuisine & Banquet" Atmosphere            1.0
                                     Food                  3.0
                                     Price                 3.0
                                     Service               2.6
...                                                        ...
"Schwartz's"                         Service               3.0
                                     Waiting time          3.3
"Spuntini Restaurant & Bar"          Atmosphere            3.4
                                     Food                  3.9
                                     Price                 2.6
                                     Service               3.5
                                     Waiting time          3.8
"Sumo Ramen"                         Atmosphere            4.3
                                     Food                  3.7
                                     Price                 3.2
                                     Service               2.4
                   